In [1]:
import pandas as pd
import random
import csv
import numpy as np
from functools import reduce

In [2]:
PATH = "../Impulso-tek/year_data"

In [3]:
def clean_int(s) -> int:
    """
    Clean integer value
    """
    if isinstance(s, str):
        s = s.strip()
        if s == "":
            return None
        else:
            return int(s)
    else:
        return s


def get_grade(df: pd.DataFrame, grade: str,VET: bool) -> pd.DataFrame:
    """
    Function that filters original data frame accordingly to the grade filter
    defined by the specific conditions
    """
    df_new = pd.DataFrame()

    if grade == "8th":
        df_new = df[
            (
                  ((df["cod_ense"] == 110) & (df["cod_grado"].isin([8])))
                | ((df["cod_ense"] == 160) & (df["cod_grado"].isin([4])))
                | ((df["cod_ense"] == 161) & (df["cod_grado"].isin([4])))
                | ((df["cod_ense"] == 163) & (df["cod_grado"].isin([4])))
               
                | ((df["cod_ense"] == 167) & (df["cod_grado"].isin([3])))
            )
        ].copy()

    if grade == "9th":
        df_new = df[
            (
                (df["cod_ense"].isin(
                        [310, 360, 361, 363, 410, 460, 463, 510, 560, 561, 563, 610,660,
                         661, 663, 710, 760, 761, 763, 810, 860, 861, 863, 910]))
                & (df["cod_grado"] == 1)
            )
        ].copy()

    if grade == "10th":
        df_new = df[
            (
                (df["cod_ense"].isin([310, 360, 410, 510, 610, 710, 810, 910]))
                & (df["cod_grado"] == 2)
            )
        ].copy()

    if grade == "11th":
        df_new = df[
            (
                (df["cod_ense"].isin(
                        [310, 360, 410, 460, 461,463, 510, 560, 561, 563, 610, 660,
                         661, 663, 710, 760, 761, 763, 810, 860, 861, 863, 910,963]))
                & (df["cod_grado"] == 3)
            )
        ].copy()
        


    if grade == "12th":
        df_new = df[
            (
                (df["cod_ense"].isin(
                        [310, 360, 410, 460, 461,463, 510, 560, 561,563, 610,660, 661,663,
                         710, 760,761, 763, 810, 860,861,863, 910,963]))
                & (df["cod_grado"] == 4)
            )
        ].copy()

    if grade == "11th12th" and VET:
        df_new = df[
            (
                (df["cod_ense"].isin(
                        [410, 460,461,463, 510,560,561, 563, 610, 660, 663, 710,760,761, 763, 810,860,861,863, 910,963]))
                & ((df["cod_grado"] == 3) | (df["cod_grado"] == 4))
            )
        ].copy()

    if grade == "11th12th" and not VET:
        df_new = df[
            (
                (df["cod_ense"].isin(
                        [361,363]))
                & (df["cod_grado"] == 3)
            )
        ].copy()


    return df_new

def read_data_all_years(
    agno: int,
    source: str,
    sep: str = ",",
    cols: str = None,
    grade: str = "",
    approved: bool = False,
    VET: bool = False,
) -> pd.DataFrame:
    """
    Reads csv of the form "{PATH}/{source}_{agno}.csv", filters columns, grades and approved 
    if specified. Returns a DataFrame with mrun identifier clean
    """
    # Read data
    df_all = pd.DataFrame()
    
    df = pd.read_csv(f"{PATH}/{source}_{agno}.csv", sep=sep)
    print(f"All data in year {str(agno)}:", df.shape[0])
    cols = cols or df.columns

    # Filters grade
    if grade and not VET:
        df = get_grade(df, grade,False)
        print(f"All data in year {str(agno)} and grade {grade}:", df.shape[0])

        # Filter approved
    if approved:
        df = df[df["sit_fin"] == "P"]
        print(f"All data in year {str(agno)} and grade {grade} that approved:", df.shape[0])
    if grade and VET:
        df = get_grade(df,grade,True)
        print(f"All data in year {agno} and grade {grade} and VET:", df.shape[0])


        # clean mrun column and removing none values
    df["mrun"] = df["mrun"].map(clean_int)
    df = df[df["mrun"].notnull()]
    df["mrun"] = df["mrun"].astype(int)
    print("Valid MRUN rows: ", df.shape[0])
        
    df_all = df_all.append(df[cols])
    print("df_all rows: ",df_all.shape[0])
    agno +=1
    return df_all


def get_statistics(baseline: pd.DataFrame, graduated: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the % and total number of students on baseline that appear on graduated.
    Returns a DataFrame by by school graduated counts 
    """   
    baseline = baseline.drop_duplicates(subset=["mrun"])
    graduated = graduated.drop_duplicates(subset=["mrun"])
    #just keep those that graduated on time
    join = baseline.merge(graduated, on='mrun',how='inner') #inner join

    new_baseline = join.groupby(['agno_y','rbd_x'])['mrun'].nunique().reset_index()   
    new_baseline.columns = ["agno","rbd","mrun"]
    return new_baseline



def get_attendance(baseline: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates atendance rate 
    Returns a DataFrame with percentage of atttendance rate
    """
    
    return baseline.groupby("agno").mean().reset_index()

def access_to_VET(students11th: pd.DataFrame, students12th: pd.DataFrame, students_11th12thVET: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates access to VET prgram at 11th or 12th grade
    Returns a DataFrame with percentage of students that access to VET program at 11th or 12th grade
    """
    unique_11th = students11th.groupby(['agno'])['mrun'].nunique().reset_index()
    unique_12th = students12th.groupby(['agno'])['mrun'].nunique().reset_index()
    unique_11th12thVET = students_11th12thVET.groupby(['agno'])['mrun'].nunique().reset_index()
    unique_11and_12th =  unique_11th.set_index('agno').add(unique_12th.set_index('agno'), fill_value=0).reset_index()
    access_to_vet_students['agno'] = unique_11th12thVET['agno']
    access_to_vet_students['VET%'] = unique_11th12thVET['mrun'] / unique_11and_12th['mrun']
    return access_to_vet_students
  

### Read data

# End-Of-Cycle

In [4]:
t = 2012
path = "../Impulso-tek/year_data"
source = "Rendimiento"

## 3. On-time graduation rate
a.number of students who graduate from 12th grade in 4 years / number of students from the 8th grade cohort

In [5]:
per_year_8th = pd.DataFrame()
totals = pd.DataFrame()

#Read data from 8th cohort
cols = ["agno", "mrun", "rbd"]
s_t_cohort = read_data_all_years(t, source, cols=cols, grade="8th", approved=False) #*
    
#Read data from those who graduated 4 years later
cols = ["agno","mrun","rbd"]
s_t_4_graduated = read_data_all_years(t+4, source, cols=cols, grade="12th", approved=True)


All data in year 2012: 3308477
All data in year 2012 and grade 8th: 272497
Valid MRUN rows:  272496
df_all rows:  272496
All data in year 2016: 3226943
All data in year 2016 and grade 12th: 200567
All data in year 2016 and grade 12th that approved: 190002
Valid MRUN rows:  190002
df_all rows:  190002


In [6]:
res = get_statistics(s_t_cohort,s_t_4_graduated) #only returns those mruns that graduated 4 years later.
res

,agno,rbd,mrun
0,2016,5,77
1,2016,8,31
2,2016,9,118
3,2016,10,40
4,2016,11,10
...,...,...,...
5991,2016,40399,8
5992,2016,40403,1
5993,2016,40422,24
5994,2016,40436,9


In [7]:
#those who were at 8th grade in t
s_t_cohort_group = s_t_cohort.groupby(['agno',"rbd"])['mrun'].nunique().reset_index()
s_t_cohort_group


,agno,rbd,mrun
0,2012,3,84
1,2012,5,93
2,2012,8,75
3,2012,9,165
4,2012,10,55
...,...,...,...
6168,2012,40414,34
6169,2012,40422,77
6170,2012,40436,18
6171,2012,40437,2


In [8]:
join_both = s_t_cohort_group.merge(res, on='rbd',how='inner') #inner join
join_both['graduation_rate']= join_both['mrun_y']/join_both['mrun_x']
join_both

,agno_x,rbd,mrun_x,agno_y,mrun_y,graduation_rate
0,2012,5,93,2016,77,0.827957
1,2012,8,75,2016,31,0.413333
2,2012,9,165,2016,118,0.715152
3,2012,10,55,2016,40,0.727273
4,2012,11,33,2016,10,0.303030
...,...,...,...,...,...,...
5991,2012,40399,33,2016,8,0.242424
5992,2012,40403,16,2016,1,0.062500
5993,2012,40422,77,2016,24,0.311688
5994,2012,40436,18,2016,9,0.500000


In [9]:
filename = str(t) + "_End_of_Cycle_3_On_Time_Graduation.csv"
join_both.to_csv(filename,index=False)

## 4. Total Graduation Rate after 3 years
a.number of students who graduate from 12th grade/ number of students from the 8th grade cohort

In [10]:
t1 = t+5
all_sub_years = pd.DataFrame() #saves info of all mruns in t+n years

#Read the year t
cols = ["agno", "mrun", "rbd", ]
s_t_cohort = read_data_all_years(t, source, cols=cols, grade="8th", approved=True)
    
while t1<= 2019: #we get info about those who eventually graduated (not on time)
        
        drop_outs = pd.DataFrame()
        cols = ['agno','rbd','mrun']
        subseq_year = read_data_all_years(t1, source, grade='12th', cols=cols, approved=True)          
        all_sub_years = all_sub_years.append(subseq_year,ignore_index=True) 
        all_sub_years.drop_duplicates(subset=["mrun"])        
        t1+=1


All data in year 2012: 3308477
All data in year 2012 and grade 8th: 272497
All data in year 2012 and grade 8th that approved: 243823
Valid MRUN rows:  243823
df_all rows:  243823
All data in year 2017: 3246824
All data in year 2017 and grade 12th: 202203
All data in year 2017 and grade 12th that approved: 191596
Valid MRUN rows:  191596
df_all rows:  191596
All data in year 2018: 3293750
All data in year 2018 and grade 12th: 207668
All data in year 2018 and grade 12th that approved: 197460
Valid MRUN rows:  197460
df_all rows:  197460
All data in year 2019: 3328915
All data in year 2019 and grade 12th: 208249
All data in year 2019 and grade 12th that approved: 198368
Valid MRUN rows:  198368
df_all rows:  198368


In [11]:
#Count how many started 
s_t_cohort.drop_duplicates(subset=["mrun"])
count_s_t_cohort = s_t_cohort.groupby(['agno','rbd'])['mrun'].nunique().reset_index() #count how many started per rbd
count_s_t_cohort

,agno,rbd,mrun
0,2012,3,29
1,2012,5,86
2,2012,8,57
3,2012,9,156
4,2012,10,53
...,...,...,...
6113,2012,40403,14
6114,2012,40414,18
6115,2012,40422,57
6116,2012,40436,17


In [12]:
#keep all that eventually graduated
all_sub_years_mrun = get_statistics(s_t_cohort,all_sub_years)
all_sub_years_mrun = all_sub_years_mrun[['rbd','mrun']].copy()
all_sub_years_mrun = all_sub_years_mrun.groupby(['rbd'])['mrun'].sum().reset_index() 
graduated = count_s_t_cohort.merge(all_sub_years_mrun, on='rbd',how='inner') #inner join
graduated

,agno,rbd,mrun_x,mrun_y
0,2012,5,86,9
1,2012,8,57,8
2,2012,9,156,16
3,2012,10,53,5
4,2012,11,27,2
...,...,...,...,...
5150,2012,40399,19,1
5151,2012,40403,14,3
5152,2012,40422,57,11
5153,2012,40436,17,1


In [13]:
graduated['%_drop_out']= graduated['mrun_y']/graduated['mrun_x']
graduated

,agno,rbd,mrun_x,mrun_y,%_drop_out
0,2012,5,86,9,0.104651
1,2012,8,57,8,0.140351
2,2012,9,156,16,0.102564
3,2012,10,53,5,0.094340
4,2012,11,27,2,0.074074
...,...,...,...,...,...
5150,2012,40399,19,1,0.052632
5151,2012,40403,14,3,0.214286
5152,2012,40422,57,11,0.192982
5153,2012,40436,17,1,0.058824


In [14]:
filename = str(t) + '_End_Of_Cycle_4_Graduted_Non_On_Time.csv'
graduated.to_csv(filename, index=False)

## 5. Average Graduation Rate
a.average graduation rate from the 8th grade cohort

In [15]:
y_graduate = 4  #years that takes them to graduate
t1 = t+ y_graduate
total_count = 0 #total graduates from t cohort
count = 0 # graduates in t1 year
years_to_graduate = 0 #count*y_graduate
sum_years_to_graduate = 0 # sum_years_to_graduate += count*y_graduate

#Read the year t
cols = ["agno", "mrun", "rbd", ]
s_t_cohort = read_data_all_years(t, source, cols=cols, grade="8th", approved=False)

#Count how many started 
s_t_cohort.drop_duplicates(subset=["mrun"])
count_s_t_cohort = s_t_cohort.groupby(['agno','rbd'])['mrun'].nunique().reset_index() #count how many started per rbd


all_sub_years  = count_s_t_cohort.copy()   
all_sub_years.rename(columns={"agno":"agno_c","mrun":"mrun_c"},inplace=True)
all_sub_years_t1 = count_s_t_cohort.copy()   #students that lasted t1 years to graduate
all_sub_years_t1.rename(columns={"agno":"agno_c","mrun":"mrun_c"},inplace=True)
columnsn =[]   
while t1<= 2019: #we get info about those who eventually graduated in time t1
        
        drop_outs = pd.DataFrame()
        cols = ['agno','rbd','mrun']
        subseq_year = read_data_all_years(t1, source, grade='12th', cols=cols, approved=True)  
        subseq_year = subseq_year.drop_duplicates(subset=['mrun'])
        
        #just kept those in subsequent years that actually were in 8th grade in t
        subseq_year_graduated = get_statistics(s_t_cohort,subseq_year)
        #we create a table where we will multiply the #students * t1-t years to graduated
        subseq_year_graduated_t1 = subseq_year_graduated.copy()
        subseq_year_graduated_t1['mrun'] = subseq_year_graduated['mrun'] * (t1-t) #years took to graduate
        all_sub_years = all_sub_years.merge(subseq_year_graduated, on='rbd',how='outer') #outer join        
        all_sub_years.rename(columns={"rbd_x":"rbd"},inplace=True)
        #students that lasted t1-t years to graduate
        all_sub_years_t1 = all_sub_years_t1.merge(subseq_year_graduated_t1, on='rbd',how='outer') #outer join
        all_sub_years_t1.rename(columns={"rbd_x":"rbd"},inplace=True)
        columnsn.append(str(t1))
        t1+=1

All data in year 2012: 3308477
All data in year 2012 and grade 8th: 272497
Valid MRUN rows:  272496
df_all rows:  272496
All data in year 2016: 3226943
All data in year 2016 and grade 12th: 200567
All data in year 2016 and grade 12th that approved: 190002
Valid MRUN rows:  190002
df_all rows:  190002
All data in year 2017: 3246824
All data in year 2017 and grade 12th: 202203
All data in year 2017 and grade 12th that approved: 191596
Valid MRUN rows:  191596
df_all rows:  191596
All data in year 2018: 3293750
All data in year 2018 and grade 12th: 207668
All data in year 2018 and grade 12th that approved: 197460
Valid MRUN rows:  197460
df_all rows:  197460
All data in year 2019: 3328915
All data in year 2019 and grade 12th: 208249
All data in year 2019 and grade 12th that approved: 198368
Valid MRUN rows:  198368
df_all rows:  198368


In [16]:
all_sub_years  #will store the total count of graduated of 12th over the years

,agno_c,rbd,mrun_c,agno_x,mrun_x,agno_y,mrun_y,agno_x,mrun_x,agno_y,mrun_y
0,2012,3,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,5,93,2016.0,77.0,2017.0,10.0,NaN,NaN,NaN,NaN
2,2012,8,75,2016.0,31.0,2017.0,8.0,2018.0,2.0,NaN,NaN
3,2012,9,165,2016.0,118.0,2017.0,15.0,2018.0,2.0,NaN,NaN
4,2012,10,55,2016.0,40.0,2017.0,3.0,2018.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6169,2012,40422,77,2016.0,24.0,2017.0,8.0,2018.0,2.0,NaN,NaN
6170,2012,40436,18,2016.0,9.0,2017.0,1.0,NaN,NaN,NaN,NaN
6171,2012,40437,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#add sum column
del all_sub_years['agno_x']
del all_sub_years['agno_y']
#del all_sub_years['agno']
all_sub_years_no_year= all_sub_years.assign(suma=all_sub_years.iloc[:,3:].sum(1))
all_sub_years_no_year

,agno_c,rbd,mrun_c,mrun_x,mrun_y,mrun_x,mrun_y,suma
0,2012,3,84,NaN,NaN,NaN,NaN,0.0
1,2012,5,93,77.0,10.0,NaN,NaN,87.0
2,2012,8,75,31.0,8.0,2.0,NaN,41.0
3,2012,9,165,118.0,15.0,2.0,NaN,135.0
4,2012,10,55,40.0,3.0,2.0,NaN,45.0
...,...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN,0.0
6169,2012,40422,77,24.0,8.0,2.0,NaN,34.0
6170,2012,40436,18,9.0,1.0,NaN,NaN,10.0
6171,2012,40437,2,NaN,NaN,NaN,NaN,0.0


In [18]:
all_sub_years_t1  #will store the total count of graduated of 12th over the years * t1-t (years to graduate)

,agno_c,rbd,mrun_c,agno_x,mrun_x,agno_y,mrun_y,agno_x,mrun_x,agno_y,mrun_y
0,2012,3,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,5,93,2016.0,308.0,2017.0,50.0,NaN,NaN,NaN,NaN
2,2012,8,75,2016.0,124.0,2017.0,40.0,2018.0,12.0,NaN,NaN
3,2012,9,165,2016.0,472.0,2017.0,75.0,2018.0,12.0,NaN,NaN
4,2012,10,55,2016.0,160.0,2017.0,15.0,2018.0,12.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6169,2012,40422,77,2016.0,96.0,2017.0,40.0,2018.0,12.0,NaN,NaN
6170,2012,40436,18,2016.0,36.0,2017.0,5.0,NaN,NaN,NaN,NaN
6171,2012,40437,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#remove years columns
del all_sub_years_t1['agno_x']
del all_sub_years_t1['agno_y']
#del all_sub_years_t1['agno']
y_graduate = 4  #years that takes them to graduate
t1 = t+ y_graduate
all_sub_years_t1  

,agno_c,rbd,mrun_c,mrun_x,mrun_y,mrun_x,mrun_y
0,2012,3,84,NaN,NaN,NaN,NaN
1,2012,5,93,308.0,50.0,NaN,NaN
2,2012,8,75,124.0,40.0,12.0,NaN
3,2012,9,165,472.0,75.0,12.0,NaN
4,2012,10,55,160.0,15.0,12.0,NaN
...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN
6169,2012,40422,77,96.0,40.0,12.0,NaN
6170,2012,40436,18,36.0,5.0,NaN,NaN
6171,2012,40437,2,NaN,NaN,NaN,NaN


In [20]:
#add sum column
all_sub_years_t1= all_sub_years_t1.assign(suma=all_sub_years_t1.iloc[:,3:].sum(1))
all_sub_years_t1

,agno_c,rbd,mrun_c,mrun_x,mrun_y,mrun_x,mrun_y,suma
0,2012,3,84,NaN,NaN,NaN,NaN,0.0
1,2012,5,93,308.0,50.0,NaN,NaN,358.0
2,2012,8,75,124.0,40.0,12.0,NaN,176.0
3,2012,9,165,472.0,75.0,12.0,NaN,559.0
4,2012,10,55,160.0,15.0,12.0,NaN,187.0
...,...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN,0.0
6169,2012,40422,77,96.0,40.0,12.0,NaN,148.0
6170,2012,40436,18,36.0,5.0,NaN,NaN,41.0
6171,2012,40437,2,NaN,NaN,NaN,NaN,0.0


In [21]:
#gets average years to graduate 
all_sub_years["avg_years"] = all_sub_years_t1['suma']/all_sub_years_no_year['suma']
all_sub_years

,agno_c,rbd,mrun_c,mrun_x,mrun_y,mrun_x,mrun_y,avg_years
0,2012,3,84,NaN,NaN,NaN,NaN,NaN
1,2012,5,93,77.0,10.0,NaN,NaN,4.114943
2,2012,8,75,31.0,8.0,2.0,NaN,4.292683
3,2012,9,165,118.0,15.0,2.0,NaN,4.140741
4,2012,10,55,40.0,3.0,2.0,NaN,4.155556
...,...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN,NaN
6169,2012,40422,77,24.0,8.0,2.0,NaN,4.352941
6170,2012,40436,18,9.0,1.0,NaN,NaN,4.100000
6171,2012,40437,2,NaN,NaN,NaN,NaN,NaN


In [22]:
columns = ['agno_c','rbd','mrun_c']
columns.extend(columnsn)
columns.extend(["avg_yrs"])
all_sub_years.columns = columns
all_sub_years

,agno_c,rbd,mrun_c,2016,2017,2018,2019,avg_yrs
0,2012,3,84,NaN,NaN,NaN,NaN,NaN
1,2012,5,93,77.0,10.0,NaN,NaN,4.114943
2,2012,8,75,31.0,8.0,2.0,NaN,4.292683
3,2012,9,165,118.0,15.0,2.0,NaN,4.140741
4,2012,10,55,40.0,3.0,2.0,NaN,4.155556
...,...,...,...,...,...,...,...,...
6168,2012,40414,34,NaN,NaN,NaN,NaN,NaN
6169,2012,40422,77,24.0,8.0,2.0,NaN,4.352941
6170,2012,40436,18,9.0,1.0,NaN,NaN,4.100000
6171,2012,40437,2,NaN,NaN,NaN,NaN,NaN


In [23]:
filename = str(t) + '_End_Of_Cycle_5_Avg_Graduation_Rate.csv'
all_sub_years.to_csv(filename, index=False)

## 6. Drop-out rate
a. average dropout rate from the 8th grade cohort

In [24]:
t1 = t+1
all_sub_years = pd.DataFrame() #saves info of all mruns in t+n years

#Read the year t
#we can only check if those on 8th grade in 2018 dropped out in 2019 (we have no info about 2020 yet)
cols = ["agno", "mrun", "rbd", ]
s_t_cohort = read_data_all_years(t, source, cols=cols, grade="8th", approved=True) #*
    
while t1<= 2019: #we get info about those who were in school in t+1
        
        drop_outs = pd.DataFrame()
        cols = ['agno','rbd','mrun']
        subseq_year = read_data_all_years(t1, source, cols=cols, approved=False)  #get info about current_year
        
        all_sub_years = all_sub_years.append(subseq_year,ignore_index=True) 
        all_sub_years.drop_duplicates(subset=["mrun"])
        
        t1+=1


All data in year 2012: 3308477
All data in year 2012 and grade 8th: 272497
All data in year 2012 and grade 8th that approved: 243823
Valid MRUN rows:  243823
df_all rows:  243823
All data in year 2013: 3255518
Valid MRUN rows:  3255518
df_all rows:  3255518
All data in year 2014: 3227534
Valid MRUN rows:  3227534
df_all rows:  3227534
All data in year 2015: 3238586
Valid MRUN rows:  3238586
df_all rows:  3238586
All data in year 2016: 3226943
Valid MRUN rows:  3226943
df_all rows:  3226943
All data in year 2017: 3246824
Valid MRUN rows:  3246824
df_all rows:  3246824
All data in year 2018: 3293750
Valid MRUN rows:  3293750
df_all rows:  3293750
All data in year 2019: 3328915
Valid MRUN rows:  3328915
df_all rows:  3328915


In [25]:
#Count how many started 
s_t_cohort.drop_duplicates(subset=["mrun"])
count_s_t_cohort = s_t_cohort.groupby(['agno','rbd'])['mrun'].nunique().reset_index() #count how many started per rbd
count_s_t_cohort

,agno,rbd,mrun
0,2012,3,29
1,2012,5,86
2,2012,8,57
3,2012,9,156
4,2012,10,53
...,...,...,...
6113,2012,40403,14
6114,2012,40414,18
6115,2012,40422,57
6116,2012,40436,17


In [26]:
#keep all that eventually graduated
all_sub_years_mrun = get_statistics(s_t_cohort,all_sub_years)
all_sub_years_mrun = all_sub_years_mrun[['rbd','mrun']].copy()
all_sub_years_mrun = all_sub_years_mrun.groupby(['rbd'])['mrun'].sum().reset_index() 

#get only those that equal to rbd
drop_out = count_s_t_cohort.merge(all_sub_years_mrun, on='rbd',how='inner') #inner join
drop_out

,agno,rbd,mrun_x,mrun_y
0,2012,3,29,7
1,2012,5,86,84
2,2012,8,57,56
3,2012,9,156,156
4,2012,10,53,52
...,...,...,...,...
6108,2012,40403,14,14
6109,2012,40414,18,17
6110,2012,40422,57,57
6111,2012,40436,17,17


In [27]:
values = {"mrun_y": 0}
drop_out.fillna(value=values, inplace=True)
drop_out

,agno,rbd,mrun_x,mrun_y
0,2012,3,29,7
1,2012,5,86,84
2,2012,8,57,56
3,2012,9,156,156
4,2012,10,53,52
...,...,...,...,...
6108,2012,40403,14,14
6109,2012,40414,18,17
6110,2012,40422,57,57
6111,2012,40436,17,17


In [28]:
drop_out['%_drop_out']= 1-drop_out['mrun_y']/drop_out['mrun_x']
drop_out

,agno,rbd,mrun_x,mrun_y,%_drop_out
0,2012,3,29,7,0.758621
1,2012,5,86,84,0.023256
2,2012,8,57,56,0.017544
3,2012,9,156,156,0.000000
4,2012,10,53,52,0.018868
...,...,...,...,...,...
6108,2012,40403,14,14,0.000000
6109,2012,40414,18,17,0.055556
6110,2012,40422,57,57,0.000000
6111,2012,40436,17,17,0.000000


In [29]:
filename = str(t) + '_End_Of_Cycle_6_Drop_out.csv'
drop_out.to_csv(filename, index=False)

## 7. Stop outs
number of students who stop and enter again in 3 years/number of students from the 8th grade cohort

i. Exit: leaves in year n and comes back in year n+1 or n+2

In [30]:
#Example
# Leaves in 2008 but not 2009 but returns in 2010 or 2011
totals_c = pd.DataFrame()
per_year_8th_c = pd.DataFrame()

#Read data from 8th cohort
cols = ["agno", "mrun", "rbd"]
s_t_cohort = read_data_all_years(t, source, cols=cols, grade="8th", approved=False)    
    
#those students that are in 9th grade in t+2 or t+3 
cols = ["agno","mrun","rbd"]
s_t_4_graduated_t2 = read_data_all_years(t+2, source, cols=cols, grade="9th", approved=True)         
s_t_4_graduated_t3 = read_data_all_years(t+3, source, cols=cols, grade="9th", approved=True)  
    
    
#we combine both results, and drop duplicates
all_s_t2_t3 = pd.concat([s_t_4_graduated_t2,s_t_4_graduated_t3],axis=0)
all_s_t2_t3 = all_s_t2_t3.drop_duplicates(subset=["mrun"])

    
#We do the join with 8th grade so only those that were in 8th grade in t remain
res_c = get_statistics(s_t_cohort,all_s_t2_t3) #only returns those mruns that were in 9th in t+2 or t+3

    
#Count how many started in 8th cohort     
data8th_c  = s_t_cohort.groupby(['agno','rbd'])['mrun'].nunique().reset_index() 

All data in year 2012: 3308477
All data in year 2012 and grade 8th: 272497
Valid MRUN rows:  272496
df_all rows:  272496
All data in year 2014: 3227534
All data in year 2014 and grade 9th: 363861
All data in year 2014 and grade 9th that approved: 252410
Valid MRUN rows:  252410
df_all rows:  252410
All data in year 2015: 3238586
All data in year 2015 and grade 9th: 367501
All data in year 2015 and grade 9th that approved: 255568
Valid MRUN rows:  255568
df_all rows:  255568


In [31]:
#join thos we were in 9th in t+2 and t+3 with those who started in t at 8th grade
stop_out = data8th_c.merge(res_c, on='rbd',how='inner') #inner join
stop_out['%_stop_outs']= stop_out['mrun_y']/stop_out['mrun_x']
stop_out


,agno_x,rbd,mrun_x,agno_y,mrun_y,%_stop_outs
0,2012,3,84,2014,2,0.023810
1,2012,5,93,2014,7,0.075269
2,2012,5,93,2015,3,0.032258
3,2012,8,75,2014,17,0.226667
4,2012,8,75,2015,4,0.053333
...,...,...,...,...,...,...
9086,2012,40422,77,2014,13,0.168831
9087,2012,40422,77,2015,2,0.025974
9088,2012,40436,18,2015,1,0.055556
9089,2012,40455,14,2014,2,0.142857


In [32]:
filename = str(t) + "_End_of_Cycle_7_Stop_Outs.csv"
stop_out.to_csv(filename,index=False)